In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [2]:
!wget -O Data-Collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
car_df = pd.read_csv('Data-Collisions.csv')
car_df.head()

--2020-09-08 22:18:58--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘Data-Collisions.csv’

Data-Collisions.csv 100%[===================>]  70.49M  23.0MB/s    in 3.1s    

2020-09-08 22:19:02 (23.0 MB/s) - ‘Data-Collisions.csv’ saved [73917638/73917638]



/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [3]:
car_df.shape

(194673, 38)

In [4]:
car_df['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [5]:
from sklearn.utils import resample

In [6]:
car_df_1 = car_df[car_df.SEVERITYCODE ==1]
car_df_2 = car_df[car_df.SEVERITYCODE ==2]

car_df_1_sample = resample(car_df_1, replace = False, n_samples = 58188, random_state = 123)
new_df = pd.concat([car_df_1_sample, car_df_2])
new_df.SEVERITYCODE.value_counts()

2    58188
1    58188
Name: SEVERITYCODE, dtype: int64

In [7]:
car_df["WEATHER"] = car_df["WEATHER"].astype('category')
car_df["ROADCOND"] = car_df["ROADCOND"].astype('category')
car_df["LIGHTCOND"] = car_df["LIGHTCOND"].astype('category')
car_df.dtypes

SEVERITYCODE         int64
X                  float64
Y                  float64
OBJECTID             int64
INCKEY               int64
COLDETKEY            int64
REPORTNO            object
STATUS              object
ADDRTYPE            object
INTKEY             float64
LOCATION            object
EXCEPTRSNCODE       object
EXCEPTRSNDESC       object
SEVERITYCODE.1       int64
SEVERITYDESC        object
COLLISIONTYPE       object
PERSONCOUNT          int64
PEDCOUNT             int64
PEDCYLCOUNT          int64
VEHCOUNT             int64
INCDATE             object
INCDTTM             object
JUNCTIONTYPE        object
SDOT_COLCODE         int64
SDOT_COLDESC        object
INATTENTIONIND      object
UNDERINFL           object
WEATHER           category
ROADCOND          category
LIGHTCOND         category
PEDROWNOTGRNT       object
SDOTCOLNUM         float64
SPEEDING            object
ST_COLCODE          object
ST_COLDESC          object
SEGLANEKEY           int64
CROSSWALKKEY         int64
H

In [8]:
car_df["WEATHER_CAT"] = car_df["WEATHER"].cat.codes
car_df["LIGHT_CAT"] = car_df["LIGHTCOND"].cat.codes
car_df["ROAD_CAT"] = car_df["ROADCOND"].cat.codes
car_df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,WEATHER_CAT,LIGHT_CAT,ROAD_CAT
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,NaN,NaN,10,Entering at angle,0,0,N,4,5,8
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N,6,2,8
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,4323031.0,NaN,32,One parked--one moving,0,0,N,4,5,0
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,NaN,NaN,23,From same direction - all others,0,0,N,1,5,0
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,4028032.0,NaN,10,Entering at angle,0,0,N,6,5,8


In [9]:
X = np.asarray(car_df [['ROAD_CAT', 'WEATHER_CAT', 'LIGHT_CAT']])
X [0:5]

array([[8, 4, 5],
       [8, 6, 2],
       [0, 4, 5],
       [0, 1, 5],
       [8, 6, 5]], dtype=int8)

In [10]:
y = np.asarray(car_df['SEVERITYCODE'])
y[0:5]

array([2, 1, 1, 1, 2])

In [11]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[ 1.50545441,  0.35364615,  0.3912104 ],
       [ 1.50545441,  1.04520829, -1.18714134],
       [-0.68713674,  0.35364615,  0.3912104 ],
       [-0.68713674, -0.68369706,  0.3912104 ],
       [ 1.50545441,  1.04520829,  0.3912104 ]])

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 4)
print ('Train set: ', X_train.shape, y_train.shape)
print ('Test set:', X_test.shape, y_train.shape)

Train set:  (136271, 3) (136271,)
Test set: (58402, 3) (136271,)


In [13]:
from sklearn.neighbors import KNeighborsClassifier
k = 25
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train, y_train)
neigh
yhatKNN = neigh.predict(X_test)
yhatKNN[0:5]

array([1, 1, 1, 1, 1])

In [15]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [16]:
jaccard_similarity_score(y_test,yhatKNN)

0.7034005684736824

In [17]:
f1_score(y_test, yhatKNN, average = 'macro')

0.41293902414507144

In [19]:
from sklearn.tree import DecisionTreeClassifier
carTree = DecisionTreeClassifier(criterion = "entropy", max_depth = 7)
carTree
carTree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [21]:
Treeyhat = carTree.predict(X_test)
print (Treeyhat [0:5])
print (y_test [0:5])

[1 1 1 1 1]
[2 1 1 2 2]


In [22]:
jaccard_similarity_score(y_test,Treeyhat)

0.7034519365775145

In [24]:
f1_score(y_test, Treeyhat, average = 'macro')

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.4129567271447957

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
regr = LogisticRegression (C=6, solver = 'liblinear').fit(X_train, y_train)
regr

LogisticRegression(C=6, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [26]:
regryhat = regr.predict(X_test)
regryhat

array([1, 1, 1, ..., 1, 1, 1])

In [27]:
yhatprob = regr.predict_proba(X_test)
yhatprob

array([[0.72734487, 0.27265513],
       [0.67253929, 0.32746071],
       [0.67265388, 0.32734612],
       ...,
       [0.67499338, 0.32500662],
       [0.79801987, 0.20198013],
       [0.67499338, 0.32500662]])

In [28]:
jaccard_similarity_score(y_test,regryhat)

0.7034519365775145

In [29]:
f1_score(y_test, regryhat, average = 'macro')

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.4129567271447957